##### Importing all necessary library to prepare dataset directory

In [3]:
import os
import shutil
import random

### Creating Dataset with 80% of data for training and 20% for validation 

In [4]:
# creating folders of validation set
os.chdir("./dataset_gray_244/val")
for i in range(ord("A"),ord("Z")+1):
    os.mkdir(chr(i))
os.chdir("../")


In [5]:
# transfering 20% of image to validation set
for i in range(ord("A"),ord("Z")+1):
    # select random 20 image out of 100 image
    validation_set = random.sample(os.listdir(f"train/{chr(i)}"),5)
    for j in validation_set:
        shutil.move(f"train/{chr(i)}/{j}",f"val/{chr(i)}")

In [6]:
# come back to main directory
os.chdir("../")

In [7]:
# defining train and validation dataset directory
train_path = "dataset_gray_244/train"
valid_path = "dataset_gray_244/val"

##### Importing all necessary library and module for ASL ditection model training 

In [8]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input,MobileNet,decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout,Flatten
from tensorflow.keras.optimizers import Adam

#### Preparing training and validation batches

In [9]:
train_batches = ImageDataGenerator(preprocessing_function=preprocess_input,brightness_range=[0.8,1.5]).flow_from_directory(
    directory = train_path, target_size=(224,224), batch_size=10,class_mode="categorical"
)
valid_batches = ImageDataGenerator(preprocessing_function=preprocess_input,brightness_range=[0.8,1.5]).flow_from_directory(
    directory = valid_path, target_size=(224,224), batch_size=10,class_mode="categorical"
)

Found 488 images belonging to 26 classes.
Found 130 images belonging to 26 classes.


#### Building a Transfer learning model from pretrained mobilenet model 

In [6]:
# get mobile_net_v3_small model with input shape (300,300,3)
mobilenet_model = tf.keras.applications.MobileNetV2()

In [5]:
# view all layer of mobilenet model and its parameters
mobilenet_model.summary()

Model: "MobilenetV3small"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 1  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 1)  0           ['input_1[0][0]']                
                                                                                                  
 Conv (Conv2D)                  (None, 112, 112, 16  144         ['rescaling[0][0]']              
                                )                                                                 
                                                                                   

In [13]:
# add a sofmax Dense layer to mobilenet model
x = mobilenet_model.layers[-2].output


In [14]:
output = Flatten()(x)
output = Dense(320, activation='relu')(output)
output = Dropout(0.2)(output)
# output = Dense(320, activation='relu')(output)
output = Dense(26,activation="softmax")(output)

In [15]:
# Build a transfer learing model from mobilenet model
model = Model(inputs=mobilenet_model.input, outputs=output)

In [16]:
# make last 27 layer only trainable
for layer in model.layers[:-23]:
    layer.trainable = False

In [17]:
# view new transfer learning models layers
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

#### Train the model

In [18]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=1e-4),
              metrics=['accuracy'])

In [5]:
# train the model
model.fit(x=train_batches,validation_data=valid_batches ,epochs=15)

Epoch 1/15
62/62 [==============================] - 40s 557ms/step - loss: 0.1384 - accuracy: 0.9660
Epoch 2/15
62/62 [==============================] - 34s 552ms/step - loss: 0.0309 - accuracy: 0.9935
Epoch 3/15
62/62 [==============================] - 34s 548ms/step - loss: 0.0180 - accuracy: 1.0000
Epoch 4/15
62/62 [==============================] - 47s 757ms/step - loss: 0.0091 - accuracy: 1.0000
Epoch 5/15
62/62 [==============================] - 42s 672ms/step - loss: 0.0072 - accuracy: 1.0000
Epoch 6/15
62/62 [==============================] - 41s 661ms/step - loss: 0.0074 - accuracy: 1.0000
Epoch 7/15
62/62 [==============================] - 38s 617ms/step - loss: 0.0060 - accuracy: 1.0000
Epoch 8/15
62/62 [==============================] - 36s 575ms/step - loss: 0.0060 - accuracy: 1.0000
Epoch 9/15
62/62 [==============================] - 33s 531ms/step - loss: 0.0086 - accuracy: 0.9984
Epoch 10/15
62/62 [==============================] - 31s 502ms/step - loss: 0.0045 - accura

In [ ]:
# save model
model.save("model_v2_gray2.h5")

In [11]:
model = load_model("model_v2_gray2.h5")

In [13]:
import tensorflow as tf
# Get your model's predictions for the validation data
predictions = model.predict_generator(valid_batches)

# Convert the predictions to class labels
y_pred = np.argmax(predictions, axis=1)

# Get the true labels for the validation data
y_true = valid_batches.classes

# Calculate precision, recall, and F-measure using scikit-learn's classification_report function
# Compute precision
precision = tf.keras.metrics.Precision()
precision.update_state(y_true, y_pred)
precision_result = precision.result()

# Compute recall
recall = tf.keras.metrics.Recall()
recall.update_state(y_true, y_pred)
recall_result = recall.result()

# Compute F1 score (F-measure)
f1_score = 2 * (precision_result * recall_result) / (precision_result + recall_result)

# Print the results
print("Precision:", precision_result.numpy())
print("Recall:", recall_result.numpy())
print("F1 Score:", f1_score.numpy())

C:\Users\Razwan\AppData\Local\Temp\ipykernel_2000\2499026362.py:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions = model.predict_generator(valid_batches)


Precision: 0.968
Recall: 0.968
F1 Score: 0.968
